# Evaluation

In [189]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [190]:
import pandas as pd
import numpy as np
from src import preprocess as prep
from src.evaluation import pick_test_segments, generate_test_segments, evaluate_segments
from src.Video import Video
from src.Segment import Segment
from src import search
import random
import cv2

## Parameters

In [252]:
NUM_VIDEOS = 100
GRID_SIZE = 3
BINS = [180, 256]
HIST_FRAME_SKIP = 20
REFRESH = False

# vergeet gebruikte params soms dus print ze maar afentoe
def printParams():
    print('Num. Vid {} - Grid {} - Bins {} - Skip {}'.format(NUM_VIDEOS, GRID_SIZE, BINS, HIST_FRAME_SKIP))

## Load training set / generate test set

In [ ]:
printParams()
training_set = prep.load_training_set(range(1, NUM_VIDEOS+1), GRID_SIZE, BINS, HIST_FRAME_SKIP, force_refresh=REFRESH)

Num. Vid 100 - Grid 3 - Bins [180, 256] - Skip 20
Loading / processing dataset...
processing 00075

In [247]:
# Set of 100 custom fragments with duration 20sec
test_set, labels = generate_test_segments(training_set, n=100, duration=20)

In [248]:
# Print statistics
print("TRAINING SET:")
print("Num. videos:   {:d}".format( len(training_set)) )
print("Num. segments: {:d}".format( np.sum([len(video.segments) for video in training_set])) )
print("Duration:      {:,.1f} s".format( np.sum([np.sum([segment.duration() for segment in video.segments]) for video in training_set])) )
print("Num frames:      {:d}".format( np.sum([np.sum([segment.num_frames() for segment in video.segments]) for video in training_set])) )
print("Num histograms:      {:d}".format( np.sum([np.sum([len(segment.histograms) for segment in video.segments]) for video in training_set])) )

print("TEST SET:")
print("Size: {:d}".format( len(test_set) ))

TRAINING SET:
Num. videos:   20
Num. segments: 2198
Duration:      7,556.4 s
Num frames:      212462
Num histograms:      11702
TEST SET:
Size: 100


# Small manual test

In [251]:
pr = True
for i in range(1):
#     x = random.choice(range(len(test_set[i])))
    found = search.findFrame(test_set[i][0], training_set, cv2.HISTCMP_CHISQR, 2, prints= pr, warnings=pr)
    print('Found {} - Expected {}'.format(found, labels[i]))

frame_dists [866053.9181560377, 839216.8183736728, 743538.1317681198, 597607.9005473851, 370390.9907010486, 209424.88241113236, 132934.55490338325]
frame_dists [101335.40968146145]
frame_dists [100397.55938798391]
frame_dists [11095.274088241113]
frame_dists [6488.485985622898, 6952.510782826617, 4965.120886368133]
frame_dists [4224.670645650597, 4653.862610637781, 3758.110793803532]
frame_dists [2698.2772215305095]
frame_dists [2452.356009661455, 3131.2657989002723]
frame_dists [3332.7937207623468, 3172.3514613238603, 3449.490420650409]
frame_dists [3606.547397918016]
frame_dists [3881.995430859382]
frame_dists [4515.7875307563845]
frame_dists [5091.468664229024]
frame_dists [4868.752409056486]
frame_dists [5931.670779763705]
frame_dists [6483.949452637049, 6599.377497414007, 7088.024057147232, 7312.671392788964]
frame_dists [6594.289509477991, 6705.58705741568, 6498.061616673756, 4644.223711285906]
frame_dists [3023.5995558554537]
frame_dists [1754.3984636248072, 1764.9028827180878, 

frame_dists [2477.5488359823253, 2540.4070434610567, 2546.3531623029744, 2490.2033238804984, 2424.9512629426895, 2719.060080459012, 2451.0094308514144, 2384.370968805645, 2410.8445558266835, 2392.4524170678624, 2621.0781507617226]
frame_dists [2385.6463760469564, 2637.8708698806304, 2443.645254896788, 2502.4198377629755, 2555.161445624217]
frame_dists [2504.7286816593023, 2487.0468904570334, 2556.2255490338325]
frame_dists [2702.617951592693, 2566.82826617424, 2888.9323352898505, 2963.781375717206, 2489.534854580228]
frame_dists [2443.8963925344588, 2459.506166325925, 2650.9751368462703]
frame_dists [2855.913539537031, 2759.453142517971, 3001.1259645188948, 3705.8709358306405, 3904.75473191321, 3720.0813163621974]
frame_dists [3955.2036536305477, 3435.5101892765283]
frame_dists [3746.3310030996504, 3304.5101892765283, 3502.3752555562883, 3343.3481501022225]
frame_dists [3336.8194288729146]
frame_dists [2931.015762052364]
frame_dists [3050.912880036932]
frame_dists [2705.8150102222517, 

frame_dists [17537.001648750247, 17545.000065950007, 17545.000065950007]
frame_dists [17527.00659500099, 17521.011145551674, 17537.001648750247, 17521.011145551674, 17541.00059355009, 12583.439161115874, 12506.49033832355]
frame_dists [12499.954956143243, 12477.09773791466, 12553.934973290246, 12529.38079535712]
frame_dists [12585.07953571193, 12586.720042208006, 12695.285101892765, 12682.095099914264, 12690.33786190068]
frame_dists [12698.583921387588, 12706.833278374992, 12688.689045703357, 12675.50326452549, 12696.934445690167, 12696.934445690167]
frame_dists [12599.848842577327, 12601.49053617358, 12606.416408362462, 12616.271714040757, 12670.560772934115, 12729.949020642352, 12634.352107102815]
frame_dists [12644.22086658313, 12635.996570599485, 7575.082503462376, 7609.074259711139]
frame_dists [7774.7327705599155, 7934.50293477544, 8061.06153135923, 8106.431247114687, 8083.11073006661, 7956.305216645783]
frame_dists [8123.645254896788, 8150.7480709622105, 8607.161379674206, 8563.

## Run model on test set

In [204]:
for method in [cv2.HISTCMP_CORREL, cv2.HISTCMP_CHISQR, cv2.cv2.HISTCMP_INTERSECT,
               cv2.HISTCMP_BHATTACHARYYA, cv2.HISTCMP_CHISQR_ALT, cv2.HISTCMP_KL_DIV]:
    %timeit -n 10 search.findFrame(test_set[0][0], training_set, method, warnings = False)


# for ch in [[0], [1], [0, 1]]:
#     print('{}'.format(ch))
#     %timeit -n 10 search.findFrame(test_set[0], training_set, cv2.HISTCMP_CORREL, channels=ch)


164 ms ± 5.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
210 ms ± 5.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
192 ms ± 2.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
153 ms ± 545 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
205 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
207 ms ± 9.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [220]:
results = []

for i, histogram in enumerate(test_set):
    print('\rSearching segment {}/{} - Histograms {}'.format(i+1, len(test_set), len(histogram), end='', flush=True))
    
    
    
    results.append(search.findFrame(histogram[0], training_set, cv2.HISTCMP_CHISQR_ALT, 2, warnings = False))

Searching segment 1/100 - Histograms 53
Searching segment 2/100 - Histograms 43
Searching segment 3/100 - Histograms 47
Searching segment 4/100 - Histograms 48
Searching segment 5/100 - Histograms 71
Searching segment 6/100 - Histograms 59
Searching segment 7/100 - Histograms 60
Searching segment 8/100 - Histograms 65
Searching segment 9/100 - Histograms 57
Searching segment 10/100 - Histograms 61
Searching segment 11/100 - Histograms 74
Searching segment 12/100 - Histograms 53
Searching segment 13/100 - Histograms 61
Searching segment 14/100 - Histograms 66
Searching segment 15/100 - Histograms 91
Searching segment 16/100 - Histograms 79
Searching segment 17/100 - Histograms 56
Searching segment 18/100 - Histograms 70
Searching segment 19/100 - Histograms 65
Searching segment 20/100 - Histograms 41
Searching segment 21/100 - Histograms 45
Searching segment 22/100 - Histograms 46
Searching segment 23/100 - Histograms 54
Searching segment 24/100 - Histograms 64
Searching segment 25/100 

IndexError: list index out of range

## Evaluate performance

In [221]:
evaluate_segments(results, labels)

Segment evaluation:
Correct: 0
Wrong:   29
Total:   29
TPR:     0.0%
